<a href="https://colab.research.google.com/github/kool7/Lyft_Motion_Prediction_for_Autonomous_Vehicles_2020_Kaggle/blob/master/colab/multimode/Lyft_L5_V5_448x448_%2B_KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This is version 5 of the series of notebooks which I made for training CNN Model for ongoing Kaggle competiton [Lyft Motion Prediction for Autonomous Vehicles](https://www.kaggle.com/c/lyft-motion-prediction-autonomous-vehicles/notebooks). In this version I am going to use **KerasTuner** for tuning my hyperparameters. This is an experimental notebook. 

### **Tasks Performed**
* Using new version of l5kit (1.1.0)
* Using KerasTuner to find good model configuration 

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preparation

In [ ]:
import os 

os.makedirs('/content/lyft-motion-prediction-autonomous-vehicles/', exist_ok=True)
os.chdir('/content/lyft-motion-prediction-autonomous-vehicles/')

%cd '/content/lyft-motion-prediction-autonomous-vehicles/'
!pwd

In [ ]:
os.makedirs('/content/lyft-motion-prediction-autonomous-vehicles/scenes/', exist_ok=True)
os.chdir('/content/lyft-motion-prediction-autonomous-vehicles/scenes/')

!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" --header="Cookie: _ga=GA1.3.1723755215.1594569279" --header="Connection: keep-alive" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/19990/1472735/compressed/scenes.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1603187870&Signature=CPLBdNFr4NiHvvfw4ucUxCAPiwFLWgLLAd53HABERiGUNIpo9bxvJf0cWBeBrmJgCWWUfsNtSzbWsfu3Bc7Im5yz5mSk%2Bb29M1mbouNAiVJCI81zu3hyZMYJky1oYKdaA3c%2F0N98PuKuKbUhPQsnJLISm5%2Ftpyx3lLKBRZzOaChOiwULjLV9JFFzLlG9mGAZ9lAWO6I7Jt57AZ7NcmkDCwsPK2bUDxxHHkMW4C7OOAD%2F98jlLL059RVgOfUhMaCcL1KVOkO1CNx0D11B5T8R6zCJgVwy7ThzjqsL7848sCjj8QtiQeK%2BR3jefHdHPQ1CzinecyB44B8TV%2BBqf6vrTw%3D%3D&response-content-disposition=attachment%3B+filename%3Dscenes.zip" -c -O 'scenes.zip'
!unzip ./scenes.zip

In [ ]:
%cd '/content/lyft-motion-prediction-autonomous-vehicles/scenes/'
!pwd
!ls

In [ ]:
%rm -r test.zarr/
%rm -r sample.zarr/
%rm scenes.zip

In [ ]:
%cd ..
!pwd
!ls

In [ ]:
os.makedirs('/content/lyft-motion-prediction-autonomous-vehicles/aerial_map/', exist_ok=True)
os.chdir('/content/lyft-motion-prediction-autonomous-vehicles/aerial_map/')

!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" --header="Cookie: _ga=GA1.3.1723755215.1594569279" --header="Connection: keep-alive" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/19990/1472735/compressed/aerial_map.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1603187901&Signature=myvFYuepRzSoGfj2ikbY4BPdamn7NfmkGMQCMFyjWW9KLL2bxWlSQgEsWJjNRZGfKgUf7YYw9Tq598oRQAYsQRpIHeue%2F6skBlOZoeq5Ti1mBok96%2FNgBgQoWUZcPZWIQjDlH9L329d0HSLB%2F4TBn%2BfEgLCC9UNuUVAcEAHQZE1DSkmG2ccTff37DavIewLihhmicVal6irVwmquSDsYLEYkJ6EKHdWHunEdw7yjq6F96%2FoXasoL2wwJq%2FdIZTluBtJ97BYYwikscOb%2FO8VER0pVL3rw1CF8KbS9obHGAWFod6NMPB0KjMcpIAPYfBkBG12mfsE9R%2BCaCYdBqftHJg%3D%3D&response-content-disposition=attachment%3B+filename%3Daerial_map.zip" -c -O 'aerial_map.zip'
!unzip ./aerial_map.zip

In [ ]:
%rm -r aerial_map.zip

In [ ]:
%cd ..
!pwd
!ls

In [ ]:
os.makedirs('/content/lyft-motion-prediction-autonomous-vehicles/semantic_map/', exist_ok=True)
os.chdir('/content/lyft-motion-prediction-autonomous-vehicles/semantic_map/')

!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" --header="Cookie: _ga=GA1.3.1723755215.1594569279" --header="Connection: keep-alive" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/19990/1472735/upload/semantic_map.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1603187927&Signature=WyRPso9IuGsKCpKwB5aVt5sqlk5TJ7F8JKywaDDqyhzJe%2FkHrqLoTOanNg1Y6SHjd0OkyoL%2F0N1F%2FlPKXZAJJ0Cf6gMSRjEYUZS%2BHeadxTgttiKEAwH0UjXEXyBFlmysMX2WVMNjPjVs%2BSs6QDhbt0%2Bnd0nozRCewDefHsR3d9S1WiuciweHH8IvXPCMWXlchl9SqTebKhRjECBBfGgoMAgGKRBkylN%2FN5zFGYlyc14jdnv1Kb2%2FIr6tJ9wTvAfhENkNo4kC0VdeHHUQrWegpIw9%2BxJ27V65w4oexwU0gNEAPDjRYZKRcaXm%2BAJ0dhoP63nTIluGccLYPbZyDxnCNQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dsemantic_map.zip" -c -O 'semantic_map.zip'
!unzip ./semantic_map.zip

In [ ]:
%rm -r semantic_map.zip
%cd ..
!pwd
!ls

# Installing Dependencies

In [ ]:
# this script transports l5kit and dependencies
!pip -q install pymap3d==2.1.0 
!pip -q install protobuf==3.12.2 
!pip -q install transforms3d 
!pip -q install zarr 
!pip -q install ptable

# previous version 1.0.6
!pip -q install --no-dependencies l5kit

# keras tuner
!pip install -q -U keras-tuner

!pip install -q git+https://github.com/qubvel/classification_models.git

# Import Packages

In [ ]:
import os, gc
import numpy as np 
import pandas as pd
from tqdm import tqdm
import kerastuner as kt
from typing import Dict, Callable

import tensorflow as tf
from tensorflow import Tensor
from tensorflow.keras.layers import *
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.activations import softmax
from tensorflow.keras.optimizers import Adam
from classification_models.keras import Classifiers

# L5 Toolkit
import l5kit
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS

In [ ]:
# root directory
INPUT_DIR = '/content/lyft-motion-prediction-autonomous-vehicles/'

# Configuration

In [ ]:
cfg = {
    'format_version': 4,
    'model_params': {
        'model_architecture': 'resnet34',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },

    'raster_params': {
        'raster_size': [448, 448],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },
    
    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 64,
        'shuffle': True,
        'num_workers': 0
    },
    
    'train_params': {
        'max_num_steps': 100000,
        'verbose': 100,
    },

    'valid_data_loader': {
        'key': 'scenes/validate.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 0
    },

    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 4
    }
}

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = INPUT_DIR
dm = LocalDataManager(None)

# get config
print(cfg)

{'format_version': 4, 'model_params': {'model_architecture': 'resnet34', 'history_num_frames': 10, 'history_step_size': 1, 'history_delta_time': 0.1, 'future_num_frames': 50, 'future_step_size': 1, 'future_delta_time': 0.1}, 'raster_params': {'raster_size': [448, 448], 'pixel_size': [0.5, 0.5], 'ego_center': [0.25, 0.5], 'map_type': 'py_semantic', 'satellite_map_key': 'aerial_map/aerial_map.png', 'semantic_map_key': 'semantic_map/semantic_map.pb', 'dataset_meta_key': 'meta.json', 'filter_agents_threshold': 0.5}, 'train_data_loader': {'key': 'scenes/train.zarr', 'batch_size': 64, 'shuffle': True, 'num_workers': 0}, 'train_params': {'max_num_steps': 100000, 'verbose': 100}, 'valid_data_loader': {'key': 'scenes/validate.zarr', 'batch_size': 32, 'shuffle': False, 'num_workers': 0}, 'test_data_loader': {'key': 'scenes/test.zarr', 'batch_size': 32, 'shuffle': False, 'num_workers': 4}}


# Keras DataGenerator

In [ ]:
# rasterizercfg, dm
rasterizer = build_rasterizer(cfg, dm)

# Train dataset/dataloader
train_cfg = cfg["train_data_loader"]
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)

In [ ]:
train_loader = ImageDataGenerator()

# Loss Function

In [ ]:
def negativeLogLikelihood(gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor):

  '''
  Description
  -------------------------------
  Compute Negative log-likelihood

  Arguments:
  gt -- array of shape (time)x(2D coords)
  pred -- array of shape (modes)x(time)x(2D coords)
  confidences -- array of shape (modes) with a confidence for each mode in each sample
  avails -- array of shape (time) with the availability for each gt timestep

  Returns:
  error -- negative log-likelihood (floating number)
  '''

  assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
  batch_size, num_modes, future_len, num_coords = pred.shape
  
  assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
  assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
  assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
  assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
  
  # assert all data are valid
  assert torch.isfinite(pred).all(), "invalid value found in pred"
  assert torch.isfinite(gt).all(), "invalid value found in gt"
  assert torch.isfinite(confidences).all(), "invalid value found in confidences"
  assert torch.isfinite(avails).all(), "invalid value found in avails"

  gt = torch.unsqueeze(gt, 1) 
  avails = avails[:, None, :, None]
  error = torch.sum(((gt - pred) * avails) ** 2, dim=-1) 

  with np.errstate(divide="ignore"): 
      error = torch.log(confidences) - 0.5 * torch.sum(error, dim = -1)

  max_value, _ = error.max(dim = 1, keepdim = True) 
  error = -torch.log(torch.sum(torch.exp(error - max_value), dim = -1, keepdim=True)) - max_value
  return torch.mean(error)

# Optimizer

In [ ]:
optimizer = Adam(0.0005)

# Model

In [ ]:
def model():

  ResNet18, preprocess_input = Classifiers.get('resnet18')
  base = ResNet18((224, 224, 3), weights=None, include_top=False)

  inp = Input(shape=(224,224,25))
  conv1 = Conv2D(3,kernel_size=3,use_bias=False,padding="same")(inp)
  x = base(conv1)
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  out = Dense(100, activation='softmax',name='out',dtype='float32')(x)
  out = out * 3
  modes = 3
  model = Model(inp, out + 3)
  return model, out, modes

In [ ]:
class LyftMultiMode(Model):
  
  def __init__(self, cfg, model, mode = 3):
    super(LyftMultiMode, self).__init__()
    self.model = model
    
  def compile(self, loss, optimizer):
    super(LyftMultiMode, self).compile()
    self.loss = loss
    self.optim = optimizer

  def train_step(self, data):
    data = train_loader.flow(train_dataset,
                             batch_size = 32)
    with tf.GradientTape(persistent=True) as tape:
      inputs = data["image"]
      target_availabilities = data["target_availabilities"]
      targets = data["target_positions"]

      outputs, num_preds, num_modes = self.model(inputs, training=True)

      bs, _ = outputs.shape
      pred, confidences = tf.split(x, num_preds, axis=1)
      pred = pred.view(bs, num_modes, cfg["model_params"]["future_num_frames"], 2)
      assert confidences.shape == (bs, num_modes)
      confidences = softmax(confidences, axis=1)

      loss = loss(targets, preds, confidences, target_availabilities)

    # compute gradients
    gradients = tape.gradient(loss, self.model.trainable_variables)

    # Update weights
    self.optim.apply_gradients(zip(gradients, self.model.trainable_variables))

    return loss

In [ ]:
Lyftmodel = LyftMultiMode(cfg, model)

In [ ]:
Lyftmodel.compile(negativeLogLikelihood, optimizer)

In [ ]:
Lyftmodel.fit(train_loader.flow(train_dataset),
              batch_size=2)